# Choropleth

## Day 13 - 30 Day Map Challenge


Today's map looks at median home value in the US using data from Zillow. Specifically, the change in median home value over the last 5 years per county using Zillow's Zillow Home Value Index (ZHVI). I used two data sources for this map:

* Zillow's [home values dataset (ZHVI)](https://www.zillow.com/research/data/) - CSV
* Natural Earth's [Admin 2 - Counties](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/) - Shapefile

![](../img/choropleth/delta.png)

To follow along create a few account in [Wherobots Cloud.](https://www.wherobots.services)

In [ ]:
# All other packages are installed by default in Wherobots Cloud
pip install mapclassify

In [ ]:
from sedona.spark import *
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.utils.adapter import Adapter

In [ ]:
config = SedonaContext.builder().appName('zillowdata')\
    .config("spark.hadoop.fs.s3a.bucket.wherobots-examples-prod.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")\
    .getOrCreate()
sedona = SedonaContext.create(config)

## Natural Earth Shapefile Import

I uploaded the Shapefile _files_ to Wherobots Cloud using this structure:

![](../img/choropleth/files1.png)

Note that since a Shapefile is composed of multiple files, the S3 URL we want to use is the location of the directory, not one of the individual files:

![](../img/choropleth/files2.png)

In [ ]:
S3_URL_NE = "s3://<YOUR_S3_URL_HERE>"

In [ ]:
spatialRDD = ShapefileReader.readToGeometryRDD(sedona, S3_URL_NE)
counties_df = Adapter.toDf(spatialRDD, sedona)
counties_df.createOrReplaceTempView("counties")
counties_df.printSchema()

```
root
 |-- geometry: geometry (nullable = true)
 |-- FEATURECLA: string (nullable = true)
 |-- SCALERANK: string (nullable = true)
 |-- ADM2_CODE: string (nullable = true)
 |-- ISO_3166_2: string (nullable = true)
 |-- ISO_A2: string (nullable = true)
 |-- ADM0_SR: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- NAME_ALT: string (nullable = true)
 |-- NAME_LOCAL: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- TYPE_EN: string (nullable = true)
 |-- CODE_LOCAL: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- REGION_COD: string (nullable = true)
 |-- ABBREV: string (nullable = true)
 |-- AREA_SQKM: string (nullable = true)
 |-- SAMEASCITY: string (nullable = true)
 |-- LABELRANK: string (nullable = true)
 |-- NAME_LEN: string (nullable = true)
 |-- MAPCOLOR9: string (nullable = true)
 |-- MAPCOLOR13: string (nullable = true)
 |-- FIPS: string (nullable = true)
 |-- SOV_A3: string (nullable = true)
 |-- ADM0_A3: string (nullable = true)
 |-- ADM0_LABEL: string (nullable = true)
 |-- ADMIN: string (nullable = true)
 |-- GEONUNIT: string (nullable = true)
 |-- GU_A3: string (nullable = true)
 |-- MIN_LABEL: string (nullable = true)
 |-- MAX_LABEL: string (nullable = true)
 |-- MIN_ZOOM: string (nullable = true)
 |-- WIKIDATAID: string (nullable = true)
 |-- NE_ID: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- NAME_AR: string (nullable = true)
 |-- NAME_BN: string (nullable = true)
 |-- NAME_DE: string (nullable = true)
 |-- NAME_EL: string (nullable = true)
 |-- NAME_EN: string (nullable = true)
 |-- NAME_ES: string (nullable = true)
 |-- NAME_FA: string (nullable = true)
 |-- NAME_FR: string (nullable = true)
 |-- NAME_HE: string (nullable = true)
 |-- NAME_HI: string (nullable = true)
 |-- NAME_HU: string (nullable = true)
 |-- NAME_ID: string (nullable = true)
 |-- NAME_IT: string (nullable = true)
 |-- NAME_JA: string (nullable = true)
 |-- NAME_KO: string (nullable = true)
 |-- NAME_NL: string (nullable = true)
 |-- NAME_PL: string (nullable = true)
 |-- NAME_PT: string (nullable = true)
 |-- NAME_RU: string (nullable = true)
 |-- NAME_SV: string (nullable = true)
 |-- NAME_TR: string (nullable = true)
 |-- NAME_UK: string (nullable = true)
 |-- NAME_UR: string (nullable = true)
 |-- NAME_VI: string (nullable = true)
 |-- NAME_ZH: string (nullable = true)
 |-- NAME_ZHT: string (nullable = true)
```

In [ ]:
sedona.sql("""
SELECT geometry, FIPS AS fips, NAME_EN AS name, REGION AS state FROM counties LIMIT 10
""").show()

```
+--------------------+-------+------------+-----+
|            geometry|   fips|        name|state|
+--------------------+-------+------------+-----+
|MULTIPOLYGON (((-...|US53073|     Whatcom|   WA|
|POLYGON ((-120.85...|US53047|    Okanogan|   WA|
|POLYGON ((-118.83...|US53019|       Ferry|   WA|
|POLYGON ((-118.21...|US53065|     Stevens|   WA|
|POLYGON ((-117.42...|US53051|Pend Oreille|   WA|
|POLYGON ((-117.03...|US16021|    Boundary|   ID|
|POLYGON ((-116.04...|US30053|     Lincoln|   MT|
|POLYGON ((-114.72...|US30029|    Flathead|   MT|
|POLYGON ((-114.06...|US30035|     Glacier|   MT|
|POLYGON ((-112.19...|US30101|       Toole|   MT|
+--------------------+-------+------------+-----+
```

## Load Zillow ZHVI CSC File

Now that we have the geometries of each county we want to import the CSV file with the Zillow home value index data. I uploaded this file into Wherobots Cloud which I can access in my notebook environment using S3.

In [ ]:
S3_URL_ZHVI = "s3://<YOUR S3 URL HERE>"

In [ ]:
zhvi_df = sedona.read.format('csv').option('header','true').option('delimiter', ',').load(S3_URL_ZHVI)
zhvi_df.createOrReplaceTempView("zhvi")
zhvi_df.printSchema()

```
root
 |-- RegionID: string (nullable = true)
 |-- SizeRank: string (nullable = true)
 |-- RegionName: string (nullable = true)
 |-- RegionType: string (nullable = true)
 |-- StateName: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Metro: string (nullable = true)
 |-- StateCodeFIPS: string (nullable = true)
 |-- MunicipalCodeFIPS: string (nullable = true)
 |-- 2000-01-31: string (nullable = true)
 |-- 2000-02-29: string (nullable = true)
 |-- 2000-03-31: string (nullable = true)
 |-- 2000-04-30: string (nullable = true)
 |-- 2000-05-31: string (nullable = true)
 |-- 2000-06-30: string (nullable = true)
 |-- 2000-07-31: string (nullable = true)
 |-- 2000-08-31: string (nullable = true)
 |-- 2000-09-30: string (nullable = true)
 |-- 2000-10-31: string (nullable = true)
 |-- 2000-11-30: string (nullable = true)
 |-- 2000-12-31: string (nullable = true)
 |-- 2001-01-31: string (nullable = true)
 |-- 2001-02-28: string (nullable = true)
 |-- 2001-03-31: string (nullable = true)
 |-- 2001-04-30: string (nullable = true)
 |-- 2001-05-31: string (nullable = true)
 |-- 2001-06-30: string (nullable = true)
 |-- 2001-07-31: string (nullable = true)
 |-- 2001-08-31: string (nullable = true)
 |-- 2001-09-30: string (nullable = true)
 |-- 2001-10-31: string (nullable = true)
 |-- 2001-11-30: string (nullable = true)
 |-- 2001-12-31: string (nullable = true)
 |-- 2002-01-31: string (nullable = true)
 |-- 2002-02-28: string (nullable = true)
 |-- 2002-03-31: string (nullable = true)
 |-- 2002-04-30: string (nullable = true)
 |-- 2002-05-31: string (nullable = true)
 |-- 2002-06-30: string (nullable = true)
 |-- 2002-07-31: string (nullable = true)
 |-- 2002-08-31: string (nullable = true)
 |-- 2002-09-30: string (nullable = true)
 |-- 2002-10-31: string (nullable = true)
 |-- 2002-11-30: string (nullable = true)
 |-- 2002-12-31: string (nullable = true)
 |-- 2003-01-31: string (nullable = true)
 |-- 2003-02-28: string (nullable = true)
 |-- 2003-03-31: string (nullable = true)
 |-- 2003-04-30: string (nullable = true)
 |-- 2003-05-31: string (nullable = true)
 |-- 2003-06-30: string (nullable = true)
 |-- 2003-07-31: string (nullable = true)
 |-- 2003-08-31: string (nullable = true)
 |-- 2003-09-30: string (nullable = true)
 |-- 2003-10-31: string (nullable = true)
 |-- 2003-11-30: string (nullable = true)
 |-- 2003-12-31: string (nullable = true)
 |-- 2004-01-31: string (nullable = true)
 |-- 2004-02-29: string (nullable = true)
 |-- 2004-03-31: string (nullable = true)
 |-- 2004-04-30: string (nullable = true)
 |-- 2004-05-31: string (nullable = true)
 |-- 2004-06-30: string (nullable = true)
 |-- 2004-07-31: string (nullable = true)
 |-- 2004-08-31: string (nullable = true)
 |-- 2004-09-30: string (nullable = true)
 |-- 2004-10-31: string (nullable = true)
 |-- 2004-11-30: string (nullable = true)
 |-- 2004-12-31: string (nullable = true)
 |-- 2005-01-31: string (nullable = true)
 |-- 2005-02-28: string (nullable = true)
 |-- 2005-03-31: string (nullable = true)
 |-- 2005-04-30: string (nullable = true)
 |-- 2005-05-31: string (nullable = true)
 |-- 2005-06-30: string (nullable = true)
 |-- 2005-07-31: string (nullable = true)
 |-- 2005-08-31: string (nullable = true)
 |-- 2005-09-30: string (nullable = true)
 |-- 2005-10-31: string (nullable = true)
 |-- 2005-11-30: string (nullable = true)
 |-- 2005-12-31: string (nullable = true)
 |-- 2006-01-31: string (nullable = true)
 |-- 2006-02-28: string (nullable = true)
 |-- 2006-03-31: string (nullable = true)
 |-- 2006-04-30: string (nullable = true)
 |-- 2006-05-31: string (nullable = true)
 |-- 2006-06-30: string (nullable = true)
 |-- 2006-07-31: string (nullable = true)
 |-- 2006-08-31: string (nullable = true)
 |-- 2006-09-30: string (nullable = true)
 |-- 2006-10-31: string (nullable = true)
 |-- 2006-11-30: string (nullable = true)
 |-- 2006-12-31: string (nullable = true)
 |-- 2007-01-31: string (nullable = true)
 |-- 2007-02-28: string (nullable = true)
 |-- 2007-03-31: string (nullable = true)
 |-- 2007-04-30: string (nullable = true)
 |-- 2007-05-31: string (nullable = true)
 |-- 2007-06-30: string (nullable = true)
 |-- 2007-07-31: string (nullable = true)
 |-- 2007-08-31: string (nullable = true)
 |-- 2007-09-30: string (nullable = true)
 |-- 2007-10-31: string (nullable = true)
 |-- 2007-11-30: string (nullable = true)
 |-- 2007-12-31: string (nullable = true)
 |-- 2008-01-31: string (nullable = true)
 |-- 2008-02-29: string (nullable = true)
 |-- 2008-03-31: string (nullable = true)
 |-- 2008-04-30: string (nullable = true)
 |-- 2008-05-31: string (nullable = true)
 |-- 2008-06-30: string (nullable = true)
 |-- 2008-07-31: string (nullable = true)
 |-- 2008-08-31: string (nullable = true)
 |-- 2008-09-30: string (nullable = true)
 |-- 2008-10-31: string (nullable = true)
 |-- 2008-11-30: string (nullable = true)
 |-- 2008-12-31: string (nullable = true)
 |-- 2009-01-31: string (nullable = true)
 |-- 2009-02-28: string (nullable = true)
 |-- 2009-03-31: string (nullable = true)
 |-- 2009-04-30: string (nullable = true)
 |-- 2009-05-31: string (nullable = true)
 |-- 2009-06-30: string (nullable = true)
 |-- 2009-07-31: string (nullable = true)
 |-- 2009-08-31: string (nullable = true)
 |-- 2009-09-30: string (nullable = true)
 |-- 2009-10-31: string (nullable = true)
 |-- 2009-11-30: string (nullable = true)
 |-- 2009-12-31: string (nullable = true)
 |-- 2010-01-31: string (nullable = true)
 |-- 2010-02-28: string (nullable = true)
 |-- 2010-03-31: string (nullable = true)
 |-- 2010-04-30: string (nullable = true)
 |-- 2010-05-31: string (nullable = true)
 |-- 2010-06-30: string (nullable = true)
 |-- 2010-07-31: string (nullable = true)
 |-- 2010-08-31: string (nullable = true)
 |-- 2010-09-30: string (nullable = true)
 |-- 2010-10-31: string (nullable = true)
 |-- 2010-11-30: string (nullable = true)
 |-- 2010-12-31: string (nullable = true)
 |-- 2011-01-31: string (nullable = true)
 |-- 2011-02-28: string (nullable = true)
 |-- 2011-03-31: string (nullable = true)
 |-- 2011-04-30: string (nullable = true)
 |-- 2011-05-31: string (nullable = true)
 |-- 2011-06-30: string (nullable = true)
 |-- 2011-07-31: string (nullable = true)
 |-- 2011-08-31: string (nullable = true)
 |-- 2011-09-30: string (nullable = true)
 |-- 2011-10-31: string (nullable = true)
 |-- 2011-11-30: string (nullable = true)
 |-- 2011-12-31: string (nullable = true)
 |-- 2012-01-31: string (nullable = true)
 |-- 2012-02-29: string (nullable = true)
 |-- 2012-03-31: string (nullable = true)
 |-- 2012-04-30: string (nullable = true)
 |-- 2012-05-31: string (nullable = true)
 |-- 2012-06-30: string (nullable = true)
 |-- 2012-07-31: string (nullable = true)
 |-- 2012-08-31: string (nullable = true)
 |-- 2012-09-30: string (nullable = true)
 |-- 2012-10-31: string (nullable = true)
 |-- 2012-11-30: string (nullable = true)
 |-- 2012-12-31: string (nullable = true)
 |-- 2013-01-31: string (nullable = true)
 |-- 2013-02-28: string (nullable = true)
 |-- 2013-03-31: string (nullable = true)
 |-- 2013-04-30: string (nullable = true)
 |-- 2013-05-31: string (nullable = true)
 |-- 2013-06-30: string (nullable = true)
 |-- 2013-07-31: string (nullable = true)
 |-- 2013-08-31: string (nullable = true)
 |-- 2013-09-30: string (nullable = true)
 |-- 2013-10-31: string (nullable = true)
 |-- 2013-11-30: string (nullable = true)
 |-- 2013-12-31: string (nullable = true)
 |-- 2014-01-31: string (nullable = true)
 |-- 2014-02-28: string (nullable = true)
 |-- 2014-03-31: string (nullable = true)
 |-- 2014-04-30: string (nullable = true)
 |-- 2014-05-31: string (nullable = true)
 |-- 2014-06-30: string (nullable = true)
 |-- 2014-07-31: string (nullable = true)
 |-- 2014-08-31: string (nullable = true)
 |-- 2014-09-30: string (nullable = true)
 |-- 2014-10-31: string (nullable = true)
 |-- 2014-11-30: string (nullable = true)
 |-- 2014-12-31: string (nullable = true)
 |-- 2015-01-31: string (nullable = true)
 |-- 2015-02-28: string (nullable = true)
 |-- 2015-03-31: string (nullable = true)
 |-- 2015-04-30: string (nullable = true)
 |-- 2015-05-31: string (nullable = true)
 |-- 2015-06-30: string (nullable = true)
 |-- 2015-07-31: string (nullable = true)
 |-- 2015-08-31: string (nullable = true)
 |-- 2015-09-30: string (nullable = true)
 |-- 2015-10-31: string (nullable = true)
 |-- 2015-11-30: string (nullable = true)
 |-- 2015-12-31: string (nullable = true)
 |-- 2016-01-31: string (nullable = true)
 |-- 2016-02-29: string (nullable = true)
 |-- 2016-03-31: string (nullable = true)
 |-- 2016-04-30: string (nullable = true)
 |-- 2016-05-31: string (nullable = true)
 |-- 2016-06-30: string (nullable = true)
 |-- 2016-07-31: string (nullable = true)
 |-- 2016-08-31: string (nullable = true)
 |-- 2016-09-30: string (nullable = true)
 |-- 2016-10-31: string (nullable = true)
 |-- 2016-11-30: string (nullable = true)
 |-- 2016-12-31: string (nullable = true)
 |-- 2017-01-31: string (nullable = true)
 |-- 2017-02-28: string (nullable = true)
 |-- 2017-03-31: string (nullable = true)
 |-- 2017-04-30: string (nullable = true)
 |-- 2017-05-31: string (nullable = true)
 |-- 2017-06-30: string (nullable = true)
 |-- 2017-07-31: string (nullable = true)
 |-- 2017-08-31: string (nullable = true)
 |-- 2017-09-30: string (nullable = true)
 |-- 2017-10-31: string (nullable = true)
 |-- 2017-11-30: string (nullable = true)
 |-- 2017-12-31: string (nullable = true)
 |-- 2018-01-31: string (nullable = true)
 |-- 2018-02-28: string (nullable = true)
 |-- 2018-03-31: string (nullable = true)
 |-- 2018-04-30: string (nullable = true)
 |-- 2018-05-31: string (nullable = true)
 |-- 2018-06-30: string (nullable = true)
 |-- 2018-07-31: string (nullable = true)
 |-- 2018-08-31: string (nullable = true)
 |-- 2018-09-30: string (nullable = true)
 |-- 2018-10-31: string (nullable = true)
 |-- 2018-11-30: string (nullable = true)
 |-- 2018-12-31: string (nullable = true)
 |-- 2019-01-31: string (nullable = true)
 |-- 2019-02-28: string (nullable = true)
 |-- 2019-03-31: string (nullable = true)
 |-- 2019-04-30: string (nullable = true)
 |-- 2019-05-31: string (nullable = true)
 |-- 2019-06-30: string (nullable = true)
 |-- 2019-07-31: string (nullable = true)
 |-- 2019-08-31: string (nullable = true)
 |-- 2019-09-30: string (nullable = true)
 |-- 2019-10-31: string (nullable = true)
 |-- 2019-11-30: string (nullable = true)
 |-- 2019-12-31: string (nullable = true)
 |-- 2020-01-31: string (nullable = true)
 |-- 2020-02-29: string (nullable = true)
 |-- 2020-03-31: string (nullable = true)
 |-- 2020-04-30: string (nullable = true)
 |-- 2020-05-31: string (nullable = true)
 |-- 2020-06-30: string (nullable = true)
 |-- 2020-07-31: string (nullable = true)
 |-- 2020-08-31: string (nullable = true)
 |-- 2020-09-30: string (nullable = true)
 |-- 2020-10-31: string (nullable = true)
 |-- 2020-11-30: string (nullable = true)
 |-- 2020-12-31: string (nullable = true)
 |-- 2021-01-31: string (nullable = true)
 |-- 2021-02-28: string (nullable = true)
 |-- 2021-03-31: string (nullable = true)
 |-- 2021-04-30: string (nullable = true)
 |-- 2021-05-31: string (nullable = true)
 |-- 2021-06-30: string (nullable = true)
 |-- 2021-07-31: string (nullable = true)
 |-- 2021-08-31: string (nullable = true)
 |-- 2021-09-30: string (nullable = true)
 |-- 2021-10-31: string (nullable = true)
 |-- 2021-11-30: string (nullable = true)
 |-- 2021-12-31: string (nullable = true)
 |-- 2022-01-31: string (nullable = true)
 |-- 2022-02-28: string (nullable = true)
 |-- 2022-03-31: string (nullable = true)
 |-- 2022-04-30: string (nullable = true)
 |-- 2022-05-31: string (nullable = true)
 |-- 2022-06-30: string (nullable = true)
 |-- 2022-07-31: string (nullable = true)
 |-- 2022-08-31: string (nullable = true)
 |-- 2022-09-30: string (nullable = true)
 |-- 2022-10-31: string (nullable = true)
 |-- 2022-11-30: string (nullable = true)
 |-- 2022-12-31: string (nullable = true)
 |-- 2023-01-31: string (nullable = true)
 |-- 2023-02-28: string (nullable = true)
 |-- 2023-03-31: string (nullable = true)
 |-- 2023-04-30: string (nullable = true)
 |-- 2023-05-31: string (nullable = true)
 |-- 2023-06-30: string (nullable = true)
 |-- 2023-07-31: string (nullable = true)
 |-- 2023-08-31: string (nullable = true)
 |-- 2023-09-30: string (nullable = true)
 |-- 2023-10-31: string (nullable = true)
```

So our zillow data has information about the county and a column for each monthly home value index going back to the year 2000.

In [ ]:
sedona.sql("""
SELECT RegionName,StateName, State, Metro, StateCodeFIPS, MunicipalCodeFIPS, `2000-01-31` FROM zhvi LIMIT 10
""").show()

```
+------------------+---------+-----+--------------------+-------------+-----------------+------------------+
|        RegionName|StateName|State|               Metro|StateCodeFIPS|MunicipalCodeFIPS|        2000-01-31|
+------------------+---------+-----+--------------------+-------------+-----------------+------------------+
|Los Angeles County|       CA|   CA|Los Angeles-Long ...|           06|              037| 205974.6936348969|
|       Cook County|       IL|   IL|Chicago-Napervill...|           17|              031| 145499.5195654641|
|     Harris County|       TX|   TX|Houston-The Woodl...|           48|              201|108350.20335999562|
|   Maricopa County|       AZ|   AZ|Phoenix-Mesa-Chan...|           04|              013|143352.15409718032|
|  San Diego County|       CA|   CA|San Diego-Chula V...|           06|              073| 214883.4100763321|
|     Orange County|       CA|   CA|Los Angeles-Long ...|           06|              059|249870.60865857836|
|      Kings County|       NY|   NY|New York-Newark-J...|           36|              047|200000.03359739753|
| Miami-Dade County|       FL|   FL|Miami-Fort Lauder...|           12|              086| 119546.5261830821|
|     Dallas County|       TX|   TX|Dallas-Fort Worth...|           48|              113| 94256.23061397206|
|  Riverside County|       CA|   CA|Riverside-San Ber...|           06|              065| 149474.9873220913|
+------------------+---------+-----+--------------------+-------------+-----------------+------------------+
```

## Joining ZHVI And County Geometries

So we now have two tables `zhvi` and `counties` - let's join them together so we can plot current home value per county. To do that we'll use the FIPS code which uniquely identifes each county. The `zhvi` table break the FIPS code out into the state and municipal components so we'll need to combine then so we can join against the `counties` table.


In [ ]:
zhvi_county_df = sedona.sql("""
SELECT CAST(zhvi.`2023-10-31` AS float) AS value, zhvi.RegionName As name, counties.geometry
FROM zhvi
JOIN counties
ON CONCAT('US', zhvi.StateCodeFIPS, zhvi.MunicipalCodeFIPS) = counties.FIPS
WHERE NOT zhvi.State IN ('HI', 'AK')
""")

In [ ]:
zhvi_county_df.show(5)

```
+---------+-------------------+--------------------+
|    value|               name|            geometry|
+---------+-------------------+--------------------+
|582171.56|     Whatcom County|MULTIPOLYGON (((-...|
|300434.03|    Okanogan County|POLYGON ((-120.85...|
| 279648.9|       Ferry County|POLYGON ((-118.83...|
|365007.16|     Stevens County|POLYGON ((-118.21...|
|364851.38|Pend Oreille County|POLYGON ((-117.42...|
+---------+-------------------+--------------------+
only showing top 5 rows
```

Now that we have our home value index and the county geometry in a single DataFrame we can use a choropleth map to visualize the data geographically. The `SedonaPyDeck` package supports creating choropleths with the `create_choropleth_map` method.

In [ ]:
SedonaPyDeck.create_choropleth_map(zhvi_county_df,plot_col="value")

![](../img/choropleth/pydeck.png)

We can also use matplotlib to create a choropleth by converting our `zhvi_county_df` Sedona DataFrame to a GeoDataFrame.

In [ ]:
zhvi_gdf = geopandas.GeoDataFrame(zhvi_county_df.toPandas(), geometry="geometry")
zhvi_gdf.to_crs(epsg=3857)

In [ ]:
ax = zhvi_gdf.plot(
    column="value",
    scheme="JenksCaspall",
    cmap="YlGn",
    legend=True,
    legend_kwds={"title": "Median Home Value($)", "fmt": "{:.0f}", "loc": "lower right"},
    missing_kwds={
        "color": "lightgrey",
        "edgecolor": "red",
        "hatch": "///",
        "label": "Missing values"
    },
    figsize=(12,9)
)

ax.set_axis_off()
ax.set_title("Median Home Value By County, Oct 2023")
ax.annotate("Data from Zillow, 2023 ZHVI",(-125,25))

plt.savefig("zhvi.png",dpi=300)

![](../img/choropleth/zhvi.png)

## Visualizing Change In Home Value

So far we've plotted the current home value, let's calculate the change in home value over the last 5 years and visualize the spatial distribution.

In our SQL query let's calculate the percent change in home value from 2018 to 2023. We'll also compute the centroid of each county so we can annotate the map.


In [ ]:
delta_county_df = sedona.sql("""
SELECT 
  ((CAST(zhvi.`2023-10-31` AS float) - CAST(zhvi.`2018-10-31` AS float)) / (CAST(zhvi.`2018-10-31` AS float)) * 100 )  AS delta,
  zhvi.RegionName As name, zhvi.Statename AS state,counties.geometry, ST_Centroid(counties.geometry) AS centroid
FROM zhvi
JOIN counties
ON CONCAT('US', zhvi.StateCodeFIPS, zhvi.MunicipalCodeFIPS) = counties.FIPS
WHERE NOT zhvi.State IN ('HI', 'AK')
""")

In [ ]:
delta_county_df.show(5)

```
+-----------------+-------------------+-----+--------------------+--------------------+
|            delta|               name|state|            geometry|            centroid|
+-----------------+-------------------+-----+--------------------+--------------------+
|52.68140223225829|     Whatcom County|   WA|MULTIPOLYGON (((-...|POINT (-121.71238...|
|55.31723313387478|    Okanogan County|   WA|POLYGON ((-120.85...|POINT (-119.73730...|
|32.94520356645385|       Ferry County|   WA|POLYGON ((-118.83...|POINT (-118.51643...|
|66.86739887826734|     Stevens County|   WA|POLYGON ((-118.21...|POINT (-117.85278...|
|66.78068227229824|Pend Oreille County|   WA|POLYGON ((-117.42...|POINT (-117.27525...|
+-----------------+-------------------+-----+--------------------+--------------------+
```

In [ ]:
# Let's find the counties with the most extreme changes in home value

delta_gdf.iloc[delta_gdf['delta'].idxmax()]

```
delta                                              169.532379
name                                          Petersburg City
state                                                      VA
geometry    POLYGON ((-77.40366881899992 37.23746578695217...
centroid         POINT (-77.39189306391864 37.20426989798888)
Name: 2937, dtype: object
```

In [ ]:
delta_gdf.iloc[delta_gdf['delta'].idxmin()]

```
delta                                              -44.794506
name                                         Claiborne Parish
state                                                      LA
geometry    POLYGON ((-93.23859537841794 33.01169686180804...
centroid         POINT (-92.99522452214245 32.81998668151367)
Name: 944, dtype: object
```

In [ ]:
ax = delta_gdf.plot(
    column="delta",
    scheme="User_Defined",
    cmap="RdYlGn",
    legend=True,
    legend_kwds={"title": "Median Home Value % Change", "loc": "lower right"},
    classification_kwds={"bins":[0,20,40,60,80,100,170]},
    missing_kwds={
        "color": "lightgrey",
        #"edgecolor": "red",
        #"hatch": "///",
        "label": "Missing values"
    },
    figsize=(12,9)
)

ax.set_axis_off()
ax.set_title("Median Home Value Percent Change By County, 2018-2023")
ax.annotate("Data from Zillow's ZHVI",(-125,25))

ax.annotate('Petersburg City, VA +169.5%',
            xy=(-77.39189306391864, 37.20426989798888), xycoords='data',
            xytext=(100, 10), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='center', verticalalignment='bottom')

ax.annotate('Claiborne Parish, LA -44.8%',
            xy=(-92.99522452214245, 32.81998668151367), xycoords='data',
            xytext=(50, -110), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='center', verticalalignment='bottom')


plt.savefig("delta.png", dpi=300)

![](../img/choropleth/delta.png)

## Resources

In addition to the data sources linked at the beginning of this notebook, I found the following resources helpful when creating this map:

* [Choropleth Mapping chapter](https://geographicdata.science/book/notebooks/05_choropleth.html) from Geographic Data Science With Python book
* GeoPandas [mapping and plotting tools documentation](https://geopandas.org/en/v0.14.1/docs/user_guide/mapping.html)
* Matplotlib example of [annotating plots](https://matplotlib.org/stable/gallery/text_labels_and_annotations/annotation_demo.html)
* StackOverlow answer on [using manual breakpoints for matplotlib classes](https://stackoverflow.com/a/56695238)